## Reworking the file pathing procedure for CMIP6 models:
---

#### Objectives:
- Needed to rewrite the file path retrevial for historical and SSP245 to have a baseline to compare results to
- Needed to make the 10_12_24 ipynb file clearer - it was written in the pre christmas delusional coding spree
- learned that theres a dependance on grid choice - so need to filter those when coming up with the reduced version
- Need to the latwise calculations and alternative form of the theory so I might as well start from a good place

In [51]:
import sys
sys.path.append('/home/users/chingosa/TropExt/Functions/')
import CMIPFuncs as func
import numpy as np
import xarray as xr
import pandas as pd
import os
from pyesgf.search import SearchConnection
os.environ["ESGF_PYCLIENT_NO_FACETS_STAR_WARNING"] = "on"

def extract_varientID(string):  # also Known as extract_8_chars
    start_index = string.find('_r') 
    end_index = string.find('_', start_index+1)
    return string[start_index+1:end_index]

def extract_monType(string):  # also Known as extract_8_chars
    start_index = string.find('_') 
    end_index = string.find('_', start_index+1)
    return string[start_index+1:end_index]

def extract_SimNums(string):
    try:
        start_index = string.find('_r')
        end_index = string.find('i', start_index)
        r = int(string[start_index + 2:end_index])

        start_index = string.find('i', end_index)
        end_index = string.find('p', start_index)
        i = int(string[start_index + 1:end_index])

        start_index = string.find('p', end_index)
        end_index = string.find('f', start_index)
        p = int(string[start_index + 1:end_index])

        start_index = string.find('f', end_index)
        end_index = string.find('_', start_index)
        f = int(string[start_index + 1:end_index])
        
        return r, i, p, f
    except Exception as e:
        # Provide feedback for debugging
        print(f"Error processing string '{string}': {e}")
        return None, None, None, None
   
def getPeriod(filename):
    if 'abrupt-4xCO2' in filename: return 'abrupt-4xCO2'
    elif 'piControl' in filename: return 'piControl'
    elif 'historical' in filename: return 'historical'
    elif 'ssp245' in filename: return 'ssp245'
    
def whichVar(filename):
    if 'tas' in filename: return 'tas'
    elif 'huss' in filename: return 'huss'
    elif 'hurs' in filename: return 'hurs'
    elif 'ps' in filename: return 'ps'

def whichTimestep(filename):
    if 'day' in filename: return 'day'
    elif 'mon' in filename: return 'mon'
    else : return np.nan

def whichGrid(filename):
    varID = extract_varientID(filename)
    start_index = filename.find(varID) +len(varID)
    end_index = filename.find('_', start_index+1)
    return filename[start_index+1:end_index]

def extractDates(string):
    ncFind = string.find('.nc') 
    ncBack = string.rfind('-', 0, ncFind)
    dback = (ncFind-ncBack)-1
    stop = string[ncBack+1:ncFind]
    start = string[ncBack-dback:ncBack]
    try:
        return extractYear(start), extractYear(stop)
    except:
        print(string)
def extractYear(Date):
    return int(Date[:4])
def getExperiment(filename):
    if 'abrupt-4xCO2' in filename: return '4x'
    elif 'piControl' in filename: return '4x'
    elif 'historical' in filename: return 'SSP245'
    elif 'ssp245' in filename: return 'SSP245'

# conn = SearchConnection('https://esgf.ceda.ac.uk/esg-search', distrib=True) #UK one
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search', distrib=True) #German one
# conn = SearchConnection('https://esgf-data.dkrz.de/esg-search', distrib=True)  #US one




## This comes up with a list of agencies and models - the models list is what we will use to search
Agencies = os.listdir(f'/badc/cmip6/data/CMIP6/CMIP/')
Models = [os.listdir(f'/badc/cmip6/data/CMIP6/CMIP/{Agency}') for Agency in Agencies]
models = []
for sublist in Models:
    models.extend(sublist)

print(len(models))

def findFilePaths(model): 
    if os.path.isfile(f'/home/users/chingosa/TropExt/CMIP6_Analysis/TempData/Paths_{model}.csv'):
        print('skip')
        return None
    else:
        print(model) # for monitoring progress of parallel execution
        files = []
        for exid in ['abrupt-4xCO2','piControl','historical','ssp245']:
            for var in ['tas', 'huss', 'ps']:
                query = conn.new_context(project="CMIP6",     
                                         experiment_id=exid,
                                         source_id = model,
                                         frequency = 'day', 
                                         # member_id="r1i1p1f1",
                                         variable_id = var)

       
                results = query.search()
                for i in range(len(results)):
                    try:
                        hit = results[i].file_context().search()
                    except:
                        hit = results[i].file_context().search()
                    files += list(map(lambda f: {'model': model,
                                                 'filename': f.filename, 
                                                 'download_url': f.download_url, 
                                                 'opendap_url': f.opendap_url}, hit))
                
                if (len(results) == 0) & (var == 'ps'):
                    query = conn.new_context(project="CMIP6",     
                                         experiment_id=exid,
                                         source_id = model,
                                         frequency = 'mon', 
                                         # member_id="r1i1p1f1",
                                         variable_id = var)

       
                    results = query.search()
                    for i in range(len(results)):
                        try:
                            hit = results[i].file_context().search()
                        except:
                            hit = results[i].file_context().search()
                        files += list(map(lambda f: {'model': model,
                                                 'filename': f.filename, 
                                                 'download_url': f.download_url, 
                                                 'opendap_url': f.opendap_url}, hit))
        
        df = pd.DataFrame.from_dict(files)
        if len(df) == 0: return None
        df = df.dropna() # some opendap_urls are not found - so get rid of those
        df[['r', 'i', 'p', 'f']] = pd.DataFrame(
            df['filename'].apply(extract_SimNums).tolist(),
            index=df.index
        )
                
        df['Varient'] = df.filename.apply(extract_varientID)
        df['period'] = df.filename.apply(getPeriod)
        df['Var'] = df.filename.apply(whichVar)
        df['grid'] = df.filename.apply(whichGrid)
        df['timeStep'] = df.filename.apply(whichTimestep)
        
        # figure out the time each models cover
        df[['start', 'stop']] = df['filename'].apply(extractDates).apply(pd.Series)
        
        df.to_csv(f'/home/users/chingosa/TropExt/CMIP6_Analysis/TempData/Paths_{model}.csv')   # Saves to CSV
        return None
# func.parallel_execution(findFilePaths, models, processes=8)

72


In [ ]:
df = pd.DataFrame()
notIn = []
for model in models:
    try:
        df_add = pd.read_csv(f'TempData/Paths_{model}.csv', index_col=None)
        df_add = df_add.loc[:, ~df_add.columns.str.contains('^Unnamed')]

        
        df = pd.concat([df, df_add], ignore_index=True)
    except:
        # print(f'{model} not included')
        notIn.append(model)
# df = df.dropna() # some opendap_urls are not found - so get rid of those
df[['r', 'i', 'p', 'f']] = pd.DataFrame(
    df['filename'].apply(extract_SimNums).tolist(),
    index=df.index
)
        
df['Varient'] = df.filename.apply(extract_varientID)
df['period'] = df.filename.apply(getPeriod)
df['Var'] = df.filename.apply(whichVar)
df['grid'] = df.filename.apply(whichGrid)
df['timeStep'] = df.filename.apply(whichTimestep)

# figure out the time each models cover
df[['start', 'stop']] = df['filename'].apply(extractDates).apply(pd.Series)
df['experiment'] = df.filename.apply(getExperiment)

dfAll = df
print(len(dfAll))
print(f'we tried to find information for {len(models)}')
print(f'We are unable to find any files for {len(set(models) - set(df.model.unique())- set(notIn))} models')
print(f'We have yet to process {len(notIn)}')

insistVars = ['tas', 'huss']
allVars = ['tas', 'huss', 'ps']

dfNew = []
for experiment in ['4x', 'SSP245']:
    df = dfAll[dfAll['experiment'] == experiment]

    # some pandas magic to find models where all the insisted upon vars are included
    groups = df.groupby(['model',  'Var', 'period'], as_index=False).Var.value_counts()
    
    if len(insistVars) == 0:
        tasHuss = groups.groupby(['model'], as_index = False).model.value_counts()
    else:
        tasHuss = groups[~groups['Var'].isin(list(set(allVars) - set(insistVars)))].groupby(['model'], as_index = False).model.value_counts()
    
    goodModels = tasHuss[tasHuss['count'] == len(insistVars)*2].reset_index(drop=True).model    # the 2 in here will mess up if you have a dif number of periods
    
    print(f'we can find {len(goodModels)} models that include {insistVars} for {experiment}')
    
    df = df[df['model'].isin(goodModels)]

    shared_combinations = {}
    
    for (period, model), group in df.groupby(['period', 'model']):
        combinations_per_variable = (
            group.groupby('Var')[['r', 'f', 'p', 'i']]
            .apply(lambda var_group: set(tuple(x) for x in var_group.to_numpy()))
            .tolist()
        )
        if combinations_per_variable:
            shared_combinations_for_group = set.intersection(*combinations_per_variable)
        else:
            shared_combinations_for_group = set()
        shared_combinations[(period, model)] = shared_combinations_for_group
    
    filtered_dfs = []
    
    for (period, model), combinations in shared_combinations.items():
        if combinations:  
            first_combination = next(iter(combinations))  
            
            filtered_df = df[
                (df['period'] == period) & 
                (df['model'] == model) & 
                (df[['r', 'f', 'p', 'i']].apply(tuple, axis=1) == first_combination)
            ]
            filtered_dfs.append(filtered_df)
            
    df = pd.concat(filtered_dfs, ignore_index=True)
    df = df[df['Var'] != 'ps']    # we are going to only use monthly ps to keep things consistant
    for model in df.model.unique():
        checkEmpty = False
        for period in df.period.unique():
            if len(df[(df['model'] == model) & (df['period'] == period)]) == 0: checkEmpty = True
    
        if checkEmpty:
            df = df[~(df['model'] == model)]
    dfNew += [df]

dfNew = pd.concat(dfNew).reset_index(drop=True)

# this line removes duplicate files with the same grid
dfNew.groupby(['model', 'period', 'Var']).apply(lambda g: g[g['grid'] == g['grid'].unique()[0]]).reset_index(drop=True, inplace=True)
dfAll.to_csv('TempData/Allpaths.csv', index = None)
dfNew.to_csv('TempData/tasHusspaths.csv', index = None)

297801
we tried to find information for 72
We are unable to find any files for 0 models
We have yet to process 2
we can find 39 models that include ['tas', 'huss'] for 4x


In [53]:
# # Litness check for varient IDs
# for model in dfNew.model.unique():
#     bools = []
#     print(model)
#     for period in dfNew.period.unique():
#         print(set(dfNew[(dfNew['model'] == model) & (dfNew['period'] == period)].Varient.unique()))

# len(dfNew.model.unique())

In [54]:
#list of models where we need monthly pressure:

# getPSmodel = []
# getPSperiod = []

# for model in dfNew.model.unique():
#     for period in ['abrupt-4xCO2','piControl','historical','ssp245']:
#         df = dfAll[((dfAll['period'] == period) & (dfAll['Var'] == 'ps'))&(dfAll['model'] == model)]
#         if df.timeStep.unique() == ['day']:
#             getPSmodel += [model]
#             getPSperiod += [period]


# getPSmodel
# for i in np.arange(len(getPSmodel)):
#     files = []
#     model = getPSmodel[i]    
#     period = getPSperiod[i]
#     print(model, period)
    
#     query = conn.new_context(project="CMIP6",     
#                                          experiment_id=period,
#                                          source_id = model,
#                                          frequency = 'mon', 
#                                          variable_id = 'ps')

       
#     results = query.search()
#     for i in range(len(results)):
#         try:
#             hit = results[i].file_context().search()
#         except:
#             hit = results[i].file_context().search()
#         files += list(map(lambda f: {'model': model,
#                                      'filename': f.filename, 
#                                      'download_url': f.download_url, 
#                                      'opendap_url': f.opendap_url}, hit))
#     df = pd.DataFrame.from_dict(files)
#     dfOld = pd.read_csv(f'TempData/Paths_{model}.csv')
#     dfOld = dfOld[~((dfOld['period'] == period) & (dfOld['Var'] == 'ps'))&(dfOld['model'] == model)] # removes the daily ps
#     df = pd.concat([df, dfOld], ignore_index=True)
#     df.to_csv(f'TempData/Paths_{model}.csv', index=None)

In [55]:
# pd.set_option('display.max_rows', None)

In [56]:
# df = pd.DataFrame()
# notIn = []
# for model in models:
#     try:
#         df_add = pd.read_csv(f'TempData/Paths_{model}.csv', index_col=0)
#         df = pd.concat([df, df_add], ignore_index=True)
#     except:
#         # print(f'{model} not included')
#         notIn.append(model)
# # df = df.dropna() # some opendap_urls are not found - so get rid of those
# df[['r', 'i', 'p', 'f']] = pd.DataFrame(
#     df['filename'].apply(extract_SimNums).tolist(),
#     index=df.index
# )
        
# df['Varient'] = df.filename.apply(extract_varientID)
# df['period'] = df.filename.apply(getPeriod)
# df['Var'] = df.filename.apply(whichVar)
# df['grid'] = df.filename.apply(whichGrid)
# df['timeStep'] = df.filename.apply(whichTimestep)

# # figure out the time each models cover
# df[['start', 'stop']] = df['filename'].apply(extractDates).apply(pd.Series)
# df['experiment'] = df.filename.apply(getExperiment)
# df = df[~df['filename'].str.contains('Emon', na=False)].reset_index(drop=True)

In [57]:
# varsIn = df.groupby(['model', 'experiment', 'period'], as_index=False).Var.unique()
# varsIn = varsIn[varsIn['Var'].apply(lambda x: set(x) == set(['tas', 'huss', 'ps']))].reset_index(drop=True)

# experimentBoth = varsIn.groupby(['model', 'experiment'], as_index=False).period.nunique()
# experimentBoth[experimentBoth.period == 2].reset_index(drop=True)[['model', 'experiment']]

# filtered_experiment = experimentBoth[experimentBoth.period == 2].reset_index(drop=True)[['model', 'experiment']]

# # Step 2: Merge with the original DataFrame to filter matching rows
# filtered_df = df.merge(filtered_experiment, on=['model', 'experiment'], how='inner')
# filtered_df.groupby(['model', 'experiment'], as_index=False).nunique()

In [ ]:
## This section adds the ps data back in with the rest of it bc I'm an idiot

dfNew = pd.read_csv('TempData/tasHusspaths.csv', index_col = None)
dfAll = pd.read_csv('TempData/Allpaths.csv', index_col = None)
dfPS = dfAll[dfAll['Var'] == 'ps'].reset_index(drop=True)
dfPS = dfPS[~dfPS['filename'].str.contains('Emon|AERmon', na=False)].reset_index(drop=True)
psDF_list = []
# dfNew.merge(dfPS, on=['experiment', 'period', 'model', 'Varient', 'grid'], how='inner')
for experiment in dfNew.experiment.unique():
    expDF = dfNew[dfNew['experiment'] == experiment]
    for model in expDF.model.unique():
        for period in expDF.period.unique():
            modelDF = expDF[(expDF['model'] == model) & (expDF['period'] == period)].drop_duplicates(subset='filename', keep='first')
            Varient = modelDF.Varient.unique()
                
            modelPSDF = dfPS[((dfPS['model'] == model) & (dfPS['period'] == period))&(dfPS['Varient'] == Varient[0])].drop_duplicates(subset='filename', keep='first')
            modelPSDF['monType'] = modelPSDF.filename.apply(extract_monType)
            if len(modelPSDF['monType'].unique()) >1:
                modelPSDF = modelPSDF[modelPSDF['monType'] == modelPSDF['monType'].unique()[0]]
            modelPSDF = modelPSDF.drop(columns = ['monType'])
            psDF_list += [modelPSDF]
psDF_list = pd.concat(psDF_list, ignore_index=True)
df = pd.concat([dfNew, psDF_list], ignore_index=True)


In [ ]:
# for each variable - what is the minimum and maximum times they cover
stopMax = df.groupby(['model', 'period', 'Var'], as_index=False).stop.max()
startMin = df.groupby(['model', 'period', 'Var'], as_index=False).start.min()

# find the overlaping times shared for each model and period
stopShared = stopMax.groupby(['model', 'period'], as_index=False).min()
startShared = startMin.groupby(['model', 'period'], as_index=False).max()

span_df = pd.DataFrame({'model': startShared.model, 
              'period': startShared.period, 
              'span': stopShared.stop - startShared.start,
              'startShared': startShared.start,
              'stopShared': stopShared.stop})

# find models that don't have shared data that spans more than 20 years
insufficientSpan = []
for model in span_df.model.unique():
    spans = span_df[span_df['model'] == model].span.to_numpy()
    if any(v < 20 for v in spans):
        insufficientSpan.append(model)

print('the following models dont cover a shared 20 years: ', insufficientSpan)

# remove models with insufficient span
df = df[~df.model.isin(insufficientSpan)]

def inspan(row, padstart, padend):
    return max(row['start'], padstart) <= min(row['stop'], padend)

dfs = []  # Store results for all models and periods
i = 0 
for model in df.model.unique():
    for period in df.period.unique():
        i+=1
        # Calculate padstart and padend based on `span_df`
        mask = (span_df.model == model) & (span_df.period == period)
        if mask.sum() > 0:  # Ensure there's a match in span_df
            end = int(span_df.loc[mask, 'stopShared'].iloc[0])  # Extract the scalar value safely
            padend = end + 3
            padstart = end - 20 - 3
            
            filtered_df = df[(df.model == model) & (df.period == period)].copy()

            # Apply inspan to the filtered DataFrame
            filtered_df.loc[:, 'inspan'] = filtered_df.apply(
                lambda row: inspan(row, padstart, padend), axis=1
            )
            
            dfs.append(filtered_df)

df = pd.concat(dfs, ignore_index=True)
print(len(df.model.unique()))
df = df[df.inspan].reset_index(drop=True)
print(len(df.model.unique()))
span_df = span_df[~span_df.model.isin(insufficientSpan)].reset_index(drop = True)

df = df.drop_duplicates(subset='filename', keep='first')

## A little routine for removing files with multiple grid options
for model in df.model.unique():
    for period in df.period.unique():
        for Var in df.Var.unique():
            q = df[((df['model'] == model) & (df['period'] == period))&(df['Var'] == Var)].reset_index(drop = True)
            if len(q.grid.unique()) >1:
                q = q[q['grid'] == q.grid.unique()[0]]
            dfAdd += [q]

df = pd.concat(dfAdd, ignore_index=True)
df.to_csv('TempData/4xCO2_SSP245_reducedFilePaths.csv', index = None)
# span_df.to_csv('span_df.csv')
# df.to_csv('CO2_4x_url_reduced_wMon.csv', index=None)
